In [59]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from pandas.plotting import scatter_matrix
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn import metrics
%matplotlib inline

今回の課題の考え方を整理
-----

- Xのｃindexのc割の割当をランダムに選別 x_trainに格納
randaom.choicce()と使ってランダムに要素を抽出
- Xと同じインデックスのd列の選別しｙ_trainに格納

- 残りの10-cの値をx_testとして格納
- y_testと同じindexの値をy_testとして格納

スクラッチを参考にした資料；
https://qiita.com/kztsh/items/5afae48b2c2fd7dc20a5

In [76]:
def scratch_train_test_split(X, y, train_size=0.8, random_state=0):
    """
    検証データを分割する。
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, )
      正解値
    train_size : float (0<train_size<1)
      何割をtrainとするか指定
    Returns
    ----------
    X_train : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    X_test : 次の形のndarray, shape (n_samples, n_features)
      検証データ
    y_train : 次の形のndarray, shape (n_samples, )
      訓練データの正解値
    y_test : 次の形のndarray, shape (n_samples, )
      検証データの正解値
    """
    #ここにコードを書く
    
#     if X.shape[0] != y.shape[0]:
#         raise ValueError("X samples number({}) is not same as y {}.".format(
#                 X.shape[0], y.shape[0]))
    n_samples = X.shape[0] #Xのサンプル数を格納
    n_train = np.floor((1-train_size) * n_samples).astype(int)  #１からテストサイズの数を引いてサンプルサイズをかける。その数は切り捨ての数になる。
    n_test = n_samples - n_train #テストの数は全体のサンプル引くサンプルの数
    classes = np.unique(y)#クラスの数がそれぞれいくつあるか
    n_classes = len(classes) #クラスの数の合計
#     class_counts = np.bincount(y)
#     class_indices = np.split(np.argsort(y, kind='mergesort'),
#                              np.cumsum(class_counts)[:-1])
    
#     if shuffle is False and stratify is None:
    #シャッフルと分割
    random = np.random.RandomState(seed = random_state)
    permutation = random.permutation(n_samples)
    ind_test = permutation[:n_test]
    ind_train = permutation[n_test:(n_test + n_train)]


    X_test = X[ind_test]#テスト以下の列をすべて取得
    X_train = X[ind_train]#テストの番号からtrainの数を足した数のデータの取得
    y_test = y[ind_test]#テスト以下の列をすべて取得
    y_train = y[ind_train]#テストの番号からtrainの数を足した数のデータの取得
    
    
    #自身で作成した値
#     randam_origin = []
#     x_train = []
#     for k in range(train_size*100):#train-sizeの回数だけfor分を回す
#         x = random.choice(X)　#Xの値からランダムに行を取り出す
#         randam_origin = randam_origin.append(x)　#randam_originに対してその値をappendすることでrandam_originのリストを作成
#         x_train.append(x.loc[:, ['','']])　#randam_originから指定のカラムの情報を取り出し、x_trainに格納
        
#     x_test = 　　　　　#ｘ_trainのデータ以外のインデックスを抽出してリストの作成
#     for i in range()
    return X_train, X_test, y_train, y_test


【問題2】 分類問題を解くコードの作成
===
上記3種類の手法で3種類のデータセットを学習・推定するコードを作成してください。

データの分割
---

In [77]:
#データのインポートIris
from sklearn.datasets import load_iris
iris = load_iris()
iris.target[[10, 25, 50]]
list(data.target_names)

X = iris.data[50:]
Y = iris.target[50:]
#DataFrameに変換
# X = pd.DataFrame(data.data, columns=data.feature_names)
# Y = pd.DataFrame(data.target_names, columns=['species_name'])
# Y_target = pd.DataFrame(data.target, columns=['species'])
# df = pd.merge(X, Y_target, left_index=True , right_index=True)

In [78]:
X_train, X_test, Y_train, Y_test = scratch_train_test_split(X, Y, train_size=0.8)

In [79]:
print(X_train)
print(X_test)
print(Y_train)
print(Y_test)

[[6.3 2.3 4.4 1.3]
 [7.9 3.8 6.4 2. ]
 [5.7 2.9 4.2 1.3]
 [5.5 2.5 4.  1.3]
 [6.4 3.2 5.3 2.3]
 [6.7 2.5 5.8 1.8]
 [6.  2.2 4.  1. ]
 [6.  3.  4.8 1.8]
 [6.9 3.2 5.7 2.3]
 [6.4 3.1 5.5 1.8]
 [6.7 3.1 4.7 1.5]
 [6.1 2.8 4.  1.3]
 [6.3 2.8 5.1 1.5]
 [5.2 2.7 3.9 1.4]
 [6.3 2.5 5.  1.9]
 [7.7 3.8 6.7 2.2]
 [5.8 2.8 5.1 2.4]
 [6.2 2.9 4.3 1.3]
 [5.6 2.7 4.2 1.3]]
[[6.8 2.8 4.8 1.4]
 [6.3 3.4 5.6 2.4]
 [6.9 3.1 4.9 1.5]
 [7.6 3.  6.6 2.1]
 [7.2 3.2 6.  1.8]
 [6.8 3.2 5.9 2.3]
 [5.6 3.  4.5 1.5]
 [6.3 2.7 4.9 1.8]
 [6.5 3.  5.8 2.2]
 [6.7 3.  5.2 2.3]
 [6.3 2.9 5.6 1.8]
 [5.8 2.7 5.1 1.9]
 [6.4 2.8 5.6 2.1]
 [6.1 2.9 4.7 1.4]
 [4.9 2.4 3.3 1. ]
 [5.5 2.4 3.8 1.1]
 [6.3 2.5 4.9 1.5]
 [6.4 2.9 4.3 1.3]
 [6.  2.7 5.1 1.6]
 [6.6 2.9 4.6 1.3]
 [5.  2.3 3.3 1. ]
 [6.8 3.  5.5 2.1]
 [5.5 2.3 4.  1.3]
 [5.6 2.8 4.9 2. ]
 [5.7 3.  4.2 1.2]
 [5.1 2.5 3.  1.1]
 [6.3 3.3 4.7 1.6]
 [5.9 3.  5.1 1.8]
 [6.4 2.8 5.6 2.2]
 [6.2 2.8 4.8 1.8]
 [6.5 3.2 5.1 2. ]
 [7.4 2.8 6.1 1.9]
 [6.7 3.1 5.6 2.4]
 [7.7 2.6 6

scikit-learnを使ってロジスティック回帰:
----

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


lr = LogisticRegression()
lr.fit(X_train, Y_train)
Y_pred_regression = lr.predict(X_test)
# metrics.accuracy_score(Y_test, Y_pred_regression)

print('MESは{}'.format(mean_squared_error(Y_test, Y_pred_regression)))
print('Accuracy: {}'.format(metrics.accuracy_score(Y_test, Y_pred_regression)))

MESは0.1111111111111111
Accuracy: 0.8888888888888888


scikit-learnを使ってSVMを実装
---

In [81]:
#SVMの実装
from sklearn.svm import SVC
ｓｖm_model = SVC(gamma='scale')
ｓｖm_model.fit(X_train, Y_train)
Y_pred_svm = ｓｖm_model.predict(X_test.round().astype(int))
print('MESは{}'.format(mean_squared_error(Y_test, Y_pred_svm)))
print('Accuracy: {}'.format(metrics.accuracy_score(Y_test, Y_pred_svm)))

MESは0.13580246913580246
Accuracy: 0.8641975308641975


scikit-learnを使って決定木分析
---

In [82]:
from sklearn import tree #決定木のメソッドを呼び起こす
from sklearn.tree import DecisionTreeClassifier, export_graphviz

clf = DecisionTreeClassifier()
clf_data = clf.fit(X_train, Y_train)
y_pred_dt = clf.predict(X_test)

print('MESは{}'.format(mean_squared_error(Y_test, y_pred_dt)))
print('Accuracy: {}'.format(metrics.accuracy_score(Y_test, y_pred_dt)))

MESは0.08641975308641975
Accuracy: 0.9135802469135802


シンプルデータセット1作成コードを用いてデータ分析を行う
---

In [83]:
import numpy as np
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, int(n_samples/2))
f1 = np.random.multivariate_normal(f1, cov, int(n_samples/2))
X = np.concatenate((f0, f1))
y = np.concatenate((np.ones((int(n_samples/2))), np.ones((int(n_samples/2))) *(-1))).astype(np.int)
random_index = np.random.permutation(np.arange(n_samples))
X = X[random_index]
y = y[random_index]

データの分割：
---

In [85]:
X_train_1, X_test_1, Y_train_1, Y_test_1 = scratch_train_test_split(X, y, train_size=0.8)

In [88]:
lr = LogisticRegression()
lr.fit(X_train_1, Y_train_1)
Y_pred_regression = lr.predict(X_test_1)
# metrics.accuracy_score(Y_test, Y_pred_regression)

print('MESは{}'.format(mean_squared_error(Y_test_1, Y_pred_regression)))
print('Accuracy: {}'.format(metrics.accuracy_score(Y_test_1, Y_pred_regression)))

MESは0.0
Accuracy: 1.0


In [89]:
#SVMの実装
from sklearn.svm import SVC
ｓｖm_model = SVC(gamma='scale')
ｓｖm_model.fit(X_train_1, Y_train_1)
Y_pred_svm = ｓｖm_model.predict(X_test_1.round().astype(int))
print('MESは{}'.format(mean_squared_error(Y_test_1, Y_pred_svm)))
print('Accuracy: {}'.format(metrics.accuracy_score(Y_test_1, Y_pred_svm)))

MESは0.0
Accuracy: 1.0


In [90]:
#決定木による実装
clf = DecisionTreeClassifier()
clf_data = clf.fit(X_train_1, Y_train_1)
y_pred_dt = clf.predict(X_test_1)

print('MESは{}'.format(mean_squared_error(Y_test_1, y_pred_dt)))
print('Accuracy: {}'.format(metrics.accuracy_score(Y_test_1, y_pred_dt)))

MESは0.00997506234413965
Accuracy: 0.9975062344139651


シンプルデータセット２を用いて分析：
---

In [91]:
X_2 = np.array([[-0.44699 , -2.8073  ],[-1.4621  , -2.4586  ],
       [ 0.10645 ,  1.9242  ],[-3.5944  , -4.0112  ],
       [-0.9888  ,  4.5718  ],[-3.1625  , -3.9606  ],
       [ 0.56421 ,  0.72888 ],[-0.60216 ,  8.4636  ],
       [-0.61251 , -0.75345 ],[-0.73535 , -2.2718  ],
       [-0.80647 , -2.2135  ],[ 0.86291 ,  2.3946  ],
       [-3.1108  ,  0.15394 ],[-2.9362  ,  2.5462  ],
       [-0.57242 , -2.9915  ],[ 1.4771  ,  3.4896  ],
       [ 0.58619 ,  0.37158 ],[ 0.6017  ,  4.3439  ],
       [-2.1086  ,  8.3428  ],[-4.1013  , -4.353   ],
       [-1.9948  , -1.3927  ],[ 0.35084 , -0.031994],
       [ 0.96765 ,  7.8929  ],[-1.281   , 15.6824  ],
       [ 0.96765 , 10.083   ],[ 1.3763  ,  1.3347  ],
       [-2.234   , -2.5323  ],[-2.9452  , -1.8219  ],
       [ 0.14654 , -0.28733 ],[ 0.5461  ,  5.8245  ],
       [-0.65259 ,  9.3444  ],[ 0.59912 ,  5.3524  ],
       [ 0.50214 , -0.31818 ],[-3.0603  , -3.6461  ],
       [-6.6797  ,  0.67661 ],[-2.353   , -0.72261 ],
       [ 1.1319  ,  2.4023  ],[-0.12243 ,  9.0162  ],
       [-2.5677  , 13.1779  ],[ 0.057313,  5.4681  ]])
y_2 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

シンプルデータセット２を分割：
---

In [93]:
X_train_2, X_test_2, Y_train_2, Y_test_2 = scratch_train_test_split(X_2, y_2, train_size=0.8)

In [94]:
#ロジスティック回帰分析
lr = LogisticRegression()
lr.fit(X_train_2, Y_train_2)
Y_pred_regression = lr.predict(X_test_2)
# metrics.accuracy_score(Y_test, Y_pred_regression)

print('MESは{}'.format(mean_squared_error(Y_test_2, Y_pred_regression)))
print('Accuracy: {}'.format(metrics.accuracy_score(Y_test_2, Y_pred_regression)))

MESは0.48484848484848486
Accuracy: 0.5151515151515151


In [95]:
#SVMの実装
from sklearn.svm import SVC
ｓｖm_model = SVC(gamma='scale')
ｓｖm_model.fit(X_train_2, Y_train_2)
Y_pred_svm = ｓｖm_model.predict(X_test_2.round().astype(int))
print('MESは{}'.format(mean_squared_error(Y_test_2, Y_pred_svm)))
print('Accuracy: {}'.format(metrics.accuracy_score(Y_test_2, Y_pred_svm)))

MESは0.42424242424242425
Accuracy: 0.5757575757575758


In [96]:
clf = DecisionTreeClassifier()
clf_data = clf.fit(X_train_2, Y_train_2)
y_pred_dt = clf.predict(X_test_2)

print('MESは{}'.format(mean_squared_error(Y_test_2, y_pred_dt)))
print('Accuracy: {}'.format(metrics.accuracy_score(Y_test_2, y_pred_dt)))

MESは0.48484848484848486
Accuracy: 0.5151515151515151


【問題2】 分類問題を解くコードの作成
----
上記3種類の手法で3種類のデータセットを学習・推定するコードを作成してください。

回帰問題 : 回帰は1種類をスクラッチします。
---
線形回帰は勾配降下法を用いて計算するSGDRegressorクラスを利用してください。
sklearn.linear_model.SGDRegressor — scikit-learn 0.21.3 documentation
データセットは事前学習期間同様にHouse Pricesコンペティションのものを使います。
House Prices: Advanced Regression Techniques
train.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。

>参考にしたウェブサイト：
https://enjoyworks.jp/tech-blog/1539

下記、実装を試みるがわからなかった。。。。

In [118]:
# import numpy as np
# import matplotlib.pyplot as plt

# x  = np.array([2,4,5])
# y = np.array([1.2,2.8,5.3])


# def scratch_SGDRegressor(X, y):
#     b0, b1 = 0.0, 1.0
#     lr = 0.001
#     epochs = 10000
#     error = []

# # 10000回繰り返す
#     for epoch in range(epochs):
#     # 値の初期化
#         epoch_cost, cost_b0, cost_b1 = 0, 0, 0
    
#         for i in range(len(X)):
#         # 予測値
#             y_pred = (b0 + b1*x[i])
#         # 損失関数
#             epoch_cost += (y[i] - y_pred)**2 
        
#             for j in range(len(X)):
#             # b0,b1の偏微分
#                 partial_wrt_b0 = -2 * (y[j] - (b0 + b1*x[j]))
#                 partial_wrt_b1 = (-2 * X[j]) * (y[j] - (b0 + b1*x[j]))
            
#             # 訓練データ分の合計
#                 cost_b0 += partial_wrt_b0
#                 cost_b1 += partial_wrt_b1

#         # 係数を学習率に掛け更新
#             b0 = b0 - lr * cost_b0
#             b1 = b1 - lr * cost_b1
        
#     # エラー率を追跡 - 可視化のため
#         error.append(epoch_cost)
#     retuen
# scratch_SGDRegressor(x, y)

Skitlaernを使用して勾配降下法を用いて計算するSGDRegressorを行う。
---

In [99]:
train_df = pd.read_csv('./train.csv')
train_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [117]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split #テストデータと訓練データを分けるためのメソッド
from sklearn import metrics

#データとレーニング用とテスト用に分ける
X_train_sdg, X_test_sdg, Y_train_sdg, Y_test_sdg = train_test_split(train_df.loc[:,['GrLivArea', 'YearBuilt']], train_df['SalePrice']) #トレーニングデータとテストデータに分ける


clf_rm = linear_model.SGDRegressor(max_iter=1000)
 
# 予測モデルを作成
clf_rm.fit(X_train_sdg, Y_train_sdg)

Y_pred_sgd = clf.predict(X_test_sdg)
 
# # 回帰係数
# print(clf.coef_)
 
# # 切片 (誤差)
# print(clf.intercept_)
 
# # 決定係数
# print(clf.score(X, Y))

print('MESは{}'.format(mean_squared_error(Y_test_sdg, Y_pred_sgd)))
print('Accuracy: {}'.format(metrics.accuracy_score(Y_test_sdg, Y_pred_sgd)))

MESは540818415.4684931
Accuracy: 0.7178082191780822
